### **1. Recap: What We've Built So Far**

**Our Encoder Layer Building Progress:**

| Component | Status | Purpose |
|-----------|--------|--------|
| Multi-Head Attention | ✅ Day 7 | Relate words to each other |
| Positional Encoding | ✅ Step 1 | Add position information |
| Feed-Forward Network | ✅ Step 2 | Add non-linear transformations |
| **Layer Normalization** | 🔧 **Today** | **Stabilize training** |
| **Residual Connections** | 🔧 **Today** | **Enable gradient flow** |
| **Complete Encoder Layer** | 🔧 **Today** | **Assemble everything** |

**Today's Goal:** Combine all the pieces to build the **Complete Encoder Layer** – the fundamental building block of the Transformer Encoder!

Let's assemble everything! 🚀

### **2. The Encoder Layer Architecture**

Before we dive into the new components, let's see the complete picture:

```
                    Input (batch, seq_len, d_model)
                              |
                              ▼
                 ┌────────────────────────┐
                 │   Multi-Head Attention  │
                 └────────────────────────┘
                              |
                              ▼
                    Dropout + Residual ──────────┐
                              |                   |
                              ▼                   |
                 ┌────────────────────────┐       |
                 │    Layer Normalization  │◄─────┘
                 └────────────────────────┘
                              |
                              ▼
                 ┌────────────────────────┐
                 │   Feed-Forward Network  │
                 └────────────────────────┘
                              |
                              ▼
                    Dropout + Residual ──────────┐
                              |                   |
                              ▼                   |
                 ┌────────────────────────┐       |
                 │    Layer Normalization  │◄─────┘
                 └────────────────────────┘
                              |
                              ▼
                   Output (batch, seq_len, d_model)
```

**Key Formula (Post-Norm, Original Transformer):**

$$\text{sublayer\_output} = \text{LayerNorm}(x + \text{Sublayer}(x))$$

Where `Sublayer` is either Multi-Head Attention or Feed-Forward Network.

<div align="center">
  <img src="https://machinelearningmastery.com/wp-content/uploads/2021/08/attention_research_1.png" width="300"/>
  <p><i>The Encoder Layer: Attention + FFN with Residuals and Layer Norm</i></p>
</div>

### **3. Residual Connections (Skip Connections)**

#### **The Problem: Vanishing Gradients in Deep Networks**

As neural networks get deeper, they face a critical problem: **vanishing gradients**.

During backpropagation, gradients are multiplied through each layer:

$$\frac{\partial L}{\partial x_1} = \frac{\partial L}{\partial x_n} \cdot \frac{\partial x_n}{\partial x_{n-1}} \cdot \ldots \cdot \frac{\partial x_2}{\partial x_1}$$

If each gradient term is less than 1, the product becomes **exponentially small**:

$$0.9^{10} = 0.35 \quad \text{(10 layers)}$$
$$0.9^{50} = 0.005 \quad \text{(50 layers)}$$
$$0.9^{100} \approx 0.00003 \quad \text{(100 layers)}$$

**Result:** Early layers barely learn because their gradients are too small!

<div align="center">
  <img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*mMQ3k3L_IftXtIUJBpE6rA.png" width="500"/>
  <p><i>Vanishing gradients make deep networks hard to train</i></p>
</div>

#### **The Solution: Residual Connections**

Residual connections (introduced in ResNet) provide a **shortcut** for gradients:

$$\text{output} = x + F(x)$$

Where:
- $x$ = input (passed through unchanged)
- $F(x)$ = transformation applied by the sublayer
- $+$ = element-wise addition

**Why This Helps:**

1. **Direct Gradient Path**: Gradients can flow directly through the addition
2. **Identity Shortcut**: Even if $F(x) \approx 0$, output ≈ input (no information lost)
3. **Easier Optimization**: Network only needs to learn the "residual" $F(x)$

```
        ┌─────────────────────────┐
        │                         │
        │    Skip Connection      │
        │         (x)             │
        │                         │
   x ───┼───► [Sublayer F] ───────┼───► x + F(x)
        │         ↓               │
        │        F(x)             │
        │                         │
        └─────────────────────────┘
```

**Intuition:** Instead of learning the full transformation, the network learns what to **add** to the input. This is often easier!

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Demonstrate the gradient flow benefit of residual connections
print("=" * 60)
print("GRADIENT FLOW: With vs Without Residual Connections")
print("=" * 60)

# Simulate gradient flow through layers
num_layers = 20
gradient_factor = 0.8  # Each layer reduces gradient by this factor

# Without residual connections
gradient_no_residual = 1.0
gradients_no_res = [gradient_no_residual]
for _ in range(num_layers):
    gradient_no_residual *= gradient_factor
    gradients_no_res.append(gradient_no_residual)

# With residual connections (gradient can flow through skip connection)
# Simplified model: gradient = gradient_through_skip + gradient_through_layer
gradient_residual = 1.0
gradients_res = [gradient_residual]
for _ in range(num_layers):
    # Part through skip (preserved) + part through layer (reduced)
    gradient_residual = 0.5 * gradient_residual + 0.5 * (gradient_residual * gradient_factor)
    gradients_res.append(gradient_residual)

# Plot comparison
plt.figure(figsize=(10, 5))
plt.plot(gradients_no_res, 'r-o', label='Without Residual', markersize=4)
plt.plot(gradients_res, 'g-o', label='With Residual', markersize=4)
plt.xlabel('Layer (from output to input)', fontsize=12)
plt.ylabel('Gradient Magnitude', fontsize=12)
plt.title('Gradient Flow Through Deep Networks', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.yscale('log')
plt.show()

print(f"\nAfter {num_layers} layers:")
print(f"  Without residual: gradient = {gradients_no_res[-1]:.6f}")
print(f"  With residual:    gradient = {gradients_res[-1]:.6f}")
print(f"\n✅ Residual connections preserve {gradients_res[-1]/gradients_no_res[-1]:.1f}x more gradient!")

### **4. Layer Normalization**

#### **Why Normalize?**

Neural network activations can have wildly different scales across layers and training steps. This causes:

1. **Unstable Training**: Large activations → large gradients → exploding updates
2. **Slow Convergence**: Network spends time learning to rescale values
3. **Sensitivity to Initialization**: Bad initial weights cause training to fail

**Normalization** forces activations to have consistent statistics (mean ≈ 0, std ≈ 1), which:
- Stabilizes training
- Allows higher learning rates
- Reduces sensitivity to initialization

#### **Batch Norm vs Layer Norm**

| Aspect | Batch Normalization | Layer Normalization |
|--------|---------------------|---------------------|
| Normalizes across | Batch dimension | Feature dimension |
| Depends on | Other samples in batch | Only current sample |
| Works with small batches? | ❌ No (noisy statistics) | ✅ Yes |
| Works with variable seq lengths? | ❌ Problematic | ✅ Yes |
| Used in | CNNs (ResNet, etc.) | Transformers, RNNs |

**For Transformers, Layer Normalization is the clear choice!**

#### **Layer Normalization Formula**

For an input vector $x$ of dimension $d$:

$$\text{LayerNorm}(x) = \gamma \odot \frac{x - \mu}{\sqrt{\sigma^2 + \epsilon}} + \beta$$

Where:
- $\mu = \frac{1}{d}\sum_{i=1}^{d} x_i$ (mean across features)
- $\sigma^2 = \frac{1}{d}\sum_{i=1}^{d} (x_i - \mu)^2$ (variance across features)
- $\gamma$ = learnable scale parameter (initialized to 1)
- $\beta$ = learnable shift parameter (initialized to 0)
- $\epsilon$ = small constant for numerical stability (e.g., 1e-6)

**Visual Comparison:**

```
Batch Normalization:          Layer Normalization:
                              
  Batch                         Batch
    ↓                             ↓
┌─────────┐                   ┌─────────┐
│ ■ ■ ■ ■ │ ← Normalize       │ ■ □ □ □ │
│ ■ ■ ■ ■ │   across batch    │ ■ □ □ □ │ ← Normalize
│ ■ ■ ■ ■ │   (vertically)    │ ■ □ □ □ │   across features
└─────────┘                   └─────────┘   (horizontally)
  Features                      Features
```

In [ ]:
# Let's visualize the difference between Batch Norm and Layer Norm

# Create sample data: (batch_size=4, seq_len=3, d_model=5)
batch_size, seq_len, d_model = 4, 3, 5
x = torch.randn(batch_size, seq_len, d_model) * 3 + 2  # Mean ≈ 2, Std ≈ 3

print("Original Data Statistics")
print("=" * 50)
print(f"Shape: {x.shape} (batch={batch_size}, seq={seq_len}, features={d_model})")
print(f"Mean: {x.mean().item():.3f}")
print(f"Std:  {x.std().item():.3f}")
print()

# Layer Normalization (normalize across last dimension = features)
layer_norm = nn.LayerNorm(d_model)
x_ln = layer_norm(x)

print("After Layer Normalization")
print("=" * 50)
print(f"Shape: {x_ln.shape}")

# Check statistics for each position in each batch
print("\nPer-position statistics (should be mean≈0, std≈1):")
for b in range(min(2, batch_size)):  # Show first 2 batches
    for s in range(seq_len):
        mean = x_ln[b, s, :].mean().item()
        std = x_ln[b, s, :].std(unbiased=False).item()
        print(f"  Batch {b}, Position {s}: mean={mean:+.4f}, std={std:.4f}")

print("\n✅ Each position is normalized independently!")

In [ ]:
# Visualize Layer Normalization effect
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Sample one position from the batch
sample_before = x[0, 0, :].detach().numpy()
sample_after = x_ln[0, 0, :].detach().numpy()

# Plot 1: Before normalization
axes[0].bar(range(d_model), sample_before, color='coral', edgecolor='black')
axes[0].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[0].axhline(y=sample_before.mean(), color='blue', linestyle='--', label=f'Mean: {sample_before.mean():.2f}')
axes[0].set_xlabel('Feature Dimension')
axes[0].set_ylabel('Value')
axes[0].set_title('Before Layer Normalization')
axes[0].legend()
axes[0].set_ylim(-5, 8)

# Plot 2: After normalization
axes[1].bar(range(d_model), sample_after, color='lightgreen', edgecolor='black')
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[1].axhline(y=sample_after.mean(), color='blue', linestyle='--', label=f'Mean: {sample_after.mean():.2f}')
axes[1].set_xlabel('Feature Dimension')
axes[1].set_ylabel('Value')
axes[1].set_title('After Layer Normalization')
axes[1].legend()
axes[1].set_ylim(-5, 8)

# Plot 3: Distribution comparison
axes[2].hist(x.flatten().detach().numpy(), bins=30, alpha=0.5, label='Before', color='coral')
axes[2].hist(x_ln.flatten().detach().numpy(), bins=30, alpha=0.5, label='After', color='lightgreen')
axes[2].axvline(x=0, color='black', linestyle='-', linewidth=0.5)
axes[2].set_xlabel('Value')
axes[2].set_ylabel('Frequency')
axes[2].set_title('Distribution of All Values')
axes[2].legend()

plt.suptitle('Layer Normalization: Centering and Scaling', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("Key Observations:")
print("• Values are centered around 0 (mean ≈ 0)")
print("• Values are scaled to unit variance (std ≈ 1)")
print("• This makes training much more stable!")

### **5. Pre-Norm vs Post-Norm**

There are two common ways to apply Layer Normalization in the Encoder:

#### **Post-Norm (Original Transformer)**

$$\text{output} = \text{LayerNorm}(x + \text{Sublayer}(x))$$

```
x ──► Sublayer ──► + ──► LayerNorm ──► output
       │          ▲
       └──────────┘ (residual)
```

#### **Pre-Norm (GPT-2, Modern Transformers)**

$$\text{output} = x + \text{Sublayer}(\text{LayerNorm}(x))$$

```
x ──► LayerNorm ──► Sublayer ──► + ──► output
│                               ▲
└───────────────────────────────┘ (residual)
```

#### **Comparison**

| Aspect | Post-Norm | Pre-Norm |
|--------|-----------|----------|
| Used in | Original Transformer, BERT | GPT-2, GPT-3, LLaMA |
| Training Stability | Can be unstable | More stable |
| Gradient Flow | Gradients pass through LayerNorm | Direct gradient path |
| Learning Rate | Needs warmup | More robust |
| Final LayerNorm | Not needed | Often added at the end |

**For this tutorial, we'll use Post-Norm** (original Transformer) but mention Pre-Norm for completeness.

### **6. Building the Components**

Before we assemble the complete Encoder Layer, let's bring in the components we built in previous steps:

1. **Multi-Head Attention** (from Day 7)
2. **Positional Encoding** (from Step 1)  
3. **Position-wise Feed-Forward Network** (from Step 2)

Let's implement them here so we have everything in one place!

In [ ]:
# Component 1: Multi-Head Attention (from Day 7)

class MultiHeadAttention(nn.Module):
    """
    Multi-Head Attention mechanism.
    
    Splits the input into multiple heads, applies scaled dot-product attention
    to each head, and concatenates the results.
    
    Args:
        d_model: Model dimension (e.g., 512)
        num_heads: Number of attention heads (e.g., 8)
        dropout: Dropout probability (default: 0.1)
    """
    
    def __init__(self, d_model, num_heads, dropout=0.1):
        super(MultiHeadAttention, self).__init__()
        
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads  # Dimension per head
        
        # Linear projections for Q, K, V
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        
        # Output projection
        self.W_o = nn.Linear(d_model, d_model)
        
        self.dropout = nn.Dropout(dropout)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        """
        Compute scaled dot-product attention.
        
        Args:
            Q: Queries (batch, num_heads, seq_len, d_k)
            K: Keys (batch, num_heads, seq_len, d_k)
            V: Values (batch, num_heads, seq_len, d_k)
            mask: Optional mask (batch, 1, 1, seq_len) or (batch, 1, seq_len, seq_len)
        
        Returns:
            output: Attention output (batch, num_heads, seq_len, d_k)
            attention_weights: Attention weights (batch, num_heads, seq_len, seq_len)
        """
        d_k = Q.size(-1)
        
        # Compute attention scores: Q @ K^T / sqrt(d_k)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / np.sqrt(d_k)
        
        # Apply mask (if provided)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))
        
        # Softmax to get attention weights
        attention_weights = F.softmax(scores, dim=-1)
        attention_weights = self.dropout(attention_weights)
        
        # Apply attention to values
        output = torch.matmul(attention_weights, V)
        
        return output, attention_weights
    
    def forward(self, query, key, value, mask=None):
        """
        Forward pass of Multi-Head Attention.
        
        Args:
            query: Query tensor (batch, seq_len, d_model)
            key: Key tensor (batch, seq_len, d_model)
            value: Value tensor (batch, seq_len, d_model)
            mask: Optional attention mask
        
        Returns:
            output: Attention output (batch, seq_len, d_model)
            attention_weights: Attention weights (batch, num_heads, seq_len, seq_len)
        """
        batch_size = query.size(0)
        
        # Linear projections
        Q = self.W_q(query)  # (batch, seq_len, d_model)
        K = self.W_k(key)
        V = self.W_v(value)
        
        # Reshape for multi-head: (batch, seq_len, d_model) -> (batch, num_heads, seq_len, d_k)
        Q = Q.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = K.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = V.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        
        # Apply attention
        attn_output, attention_weights = self.scaled_dot_product_attention(Q, K, V, mask)
        
        # Concatenate heads: (batch, num_heads, seq_len, d_k) -> (batch, seq_len, d_model)
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        
        # Final linear projection
        output = self.W_o(attn_output)
        
        return output, attention_weights

print("✅ MultiHeadAttention class defined!")

In [ ]:
# Component 2: Positional Encoding (from Step 1)

class PositionalEncoding(nn.Module):
    """
    Positional Encoding using sine and cosine functions.
    
    Adds positional information to input embeddings so the model
    can distinguish between different positions in the sequence.
    
    Args:
        d_model: Embedding dimension (must match input embedding size)
        max_seq_len: Maximum sequence length to support
        dropout: Dropout probability (default: 0.1)
    """
    
    def __init__(self, d_model, max_seq_len=5000, dropout=0.1):
        super(PositionalEncoding, self).__init__()
        
        self.dropout = nn.Dropout(p=dropout)
        
        # Create positional encoding matrix
        pe = torch.zeros(max_seq_len, d_model)
        
        # Position indices: [0, 1, 2, ..., max_seq_len-1]
        position = torch.arange(0, max_seq_len, dtype=torch.float).unsqueeze(1)
        
        # Division term: 10000^(2i/d_model)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model)
        )
        
        # Apply sine to even indices, cosine to odd indices
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        # Add batch dimension: (1, max_seq_len, d_model)
        pe = pe.unsqueeze(0)
        
        # Register as buffer (not a parameter)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        """
        Add positional encoding to input embeddings.
        
        Args:
            x: Input tensor of shape (batch_size, seq_len, d_model)
            
        Returns:
            Tensor of same shape with positional encoding added
        """
        seq_len = x.size(1)
        x = x + self.pe[:, :seq_len, :]
        return self.dropout(x)

print("✅ PositionalEncoding class defined!")

In [ ]:
# Component 3: Position-wise Feed-Forward Network (from Step 2)

class PositionWiseFeedForward(nn.Module):
    """
    Position-wise Feed-Forward Network.
    
    A simple two-layer MLP applied independently to each position.
    
    Architecture:
        Input (d_model) → Linear → Activation → Dropout → Linear → Output (d_model)
    
    Args:
        d_model: Model dimension (e.g., 512)
        d_ff: Feed-forward hidden dimension (typically 4 * d_model = 2048)
        dropout: Dropout probability (default: 0.1)
        activation: Activation function ('relu' or 'gelu')
    """
    
    def __init__(self, d_model, d_ff, dropout=0.1, activation='relu'):
        super(PositionWiseFeedForward, self).__init__()
        
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        
        if activation == 'relu':
            self.activation = F.relu
        elif activation == 'gelu':
            self.activation = F.gelu
        else:
            raise ValueError(f"Unknown activation: {activation}")
    
    def forward(self, x):
        """
        Forward pass of the FFN.
        
        Args:
            x: Input tensor of shape (batch_size, seq_len, d_model)
        
        Returns:
            Output tensor of shape (batch_size, seq_len, d_model)
        """
        x = self.linear1(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

print("✅ PositionWiseFeedForward class defined!")

### **7. The Complete Encoder Layer**

Now we have all the pieces! Let's assemble them into the complete **Encoder Layer**.

**The Encoder Layer consists of:**

1. **Multi-Head Self-Attention** sublayer
2. **Residual Connection + Layer Normalization** (after attention)
3. **Position-wise Feed-Forward Network** sublayer
4. **Residual Connection + Layer Normalization** (after FFN)

**Architecture (Post-Norm):**

```
Input x
    │
    ▼
┌─────────────────────────────────┐
│     Multi-Head Attention        │
│     (Self-Attention: Q=K=V=x)   │
└─────────────────────────────────┘
    │
    ▼
  Dropout
    │
    ├──────────────┐
    ▼              │ (residual)
   Add ◄───────────┘
    │
    ▼
  LayerNorm
    │
    ▼
┌─────────────────────────────────┐
│   Position-wise Feed-Forward    │
└─────────────────────────────────┘
    │
    ▼
  Dropout
    │
    ├──────────────┐
    ▼              │ (residual)
   Add ◄───────────┘
    │
    ▼
  LayerNorm
    │
    ▼
Output
```

In [ ]:
class EncoderLayer(nn.Module):
    """
    A single Encoder Layer of the Transformer.
    
    Consists of:
    1. Multi-Head Self-Attention with residual connection and layer norm
    2. Position-wise Feed-Forward Network with residual connection and layer norm
    
    Args:
        d_model: Model dimension (e.g., 512)
        num_heads: Number of attention heads (e.g., 8)
        d_ff: Feed-forward hidden dimension (e.g., 2048)
        dropout: Dropout probability (default: 0.1)
        activation: Activation function for FFN ('relu' or 'gelu')
    """
    
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1, activation='relu'):
        super(EncoderLayer, self).__init__()
        
        # Multi-Head Self-Attention
        self.self_attention = MultiHeadAttention(d_model, num_heads, dropout)
        
        # Position-wise Feed-Forward Network
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff, dropout, activation)
        
        # Layer Normalization (one for each sublayer)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        
        # Dropout (for residual connections)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        """
        Forward pass of the Encoder Layer.
        
        Args:
            x: Input tensor of shape (batch_size, seq_len, d_model)
            mask: Optional attention mask
        
        Returns:
            output: Encoded output of shape (batch_size, seq_len, d_model)
            attention_weights: Self-attention weights (batch, num_heads, seq_len, seq_len)
        """
        # ============================================
        # Sublayer 1: Multi-Head Self-Attention
        # ============================================
        
        # Self-attention (Q = K = V = x)
        attn_output, attention_weights = self.self_attention(x, x, x, mask)
        
        # Dropout
        attn_output = self.dropout1(attn_output)
        
        # Residual connection + Layer Normalization (Post-Norm)
        x = self.norm1(x + attn_output)
        
        # ============================================
        # Sublayer 2: Position-wise Feed-Forward
        # ============================================
        
        # Feed-forward network
        ff_output = self.feed_forward(x)
        
        # Dropout
        ff_output = self.dropout2(ff_output)
        
        # Residual connection + Layer Normalization (Post-Norm)
        output = self.norm2(x + ff_output)
        
        return output, attention_weights

print("✅ EncoderLayer class defined!")
print()
print("Architecture summary:")
print("  Input → Self-Attention → Add & Norm → FFN → Add & Norm → Output")

### **8. Testing the Encoder Layer**

In [ ]:
# Test the Encoder Layer with the original Transformer configuration

# Configuration (from "Attention Is All You Need" paper)
d_model = 512
num_heads = 8
d_ff = 2048
dropout = 0.1

# Create the encoder layer
encoder_layer = EncoderLayer(
    d_model=d_model,
    num_heads=num_heads,
    d_ff=d_ff,
    dropout=dropout,
    activation='relu'
)

print("Encoder Layer Configuration:")
print(f"  d_model: {d_model}")
print(f"  num_heads: {num_heads}")
print(f"  d_ff: {d_ff}")
print(f"  dropout: {dropout}")
print()

# Create sample input
batch_size = 2
seq_len = 10

x = torch.randn(batch_size, seq_len, d_model)
print(f"Input shape: {x.shape}")
print(f"  → (batch_size={batch_size}, seq_len={seq_len}, d_model={d_model})")

# Forward pass
encoder_layer.eval()
with torch.no_grad():
    output, attention_weights = encoder_layer(x)

print(f"\nOutput shape: {output.shape}")
print(f"  → (batch_size={batch_size}, seq_len={seq_len}, d_model={d_model})")
print(f"\nAttention weights shape: {attention_weights.shape}")
print(f"  → (batch_size={batch_size}, num_heads={num_heads}, seq_len={seq_len}, seq_len={seq_len})")
print()
print("✅ Encoder Layer works correctly!")

In [ ]:
# Visualize the attention weights
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for head in range(num_heads):
    ax = axes[head // 4, head % 4]
    
    # Get attention weights for first batch, this head
    attn = attention_weights[0, head].detach().numpy()
    
    im = ax.imshow(attn, cmap='Blues', vmin=0, vmax=0.5)
    ax.set_title(f'Head {head + 1}', fontsize=11)
    ax.set_xlabel('Key Position')
    ax.set_ylabel('Query Position')
    plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

plt.suptitle('Self-Attention Weights Across All 8 Heads\n(Encoder Layer)', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("Observations:")
print("• Each head learns different attention patterns")
print("• Some heads may focus on nearby positions (local)")
print("• Some heads may attend to specific patterns (global)")
print("• Together, they capture diverse relationships!")

### **9. Parameter Count Analysis**

In [ ]:
# Let's analyze the parameter distribution in the Encoder Layer

def count_parameters(module):
    """Count trainable parameters in a module."""
    return sum(p.numel() for p in module.parameters() if p.requires_grad)

def analyze_encoder_layer(encoder_layer):
    """Detailed breakdown of encoder layer parameters."""
    
    # Count parameters for each component
    attn_params = count_parameters(encoder_layer.self_attention)
    ffn_params = count_parameters(encoder_layer.feed_forward)
    norm1_params = count_parameters(encoder_layer.norm1)
    norm2_params = count_parameters(encoder_layer.norm2)
    total_params = count_parameters(encoder_layer)
    
    print("=" * 60)
    print("ENCODER LAYER PARAMETER BREAKDOWN")
    print("=" * 60)
    print()
    print(f"{'Component':<30} {'Parameters':>12} {'Percentage':>12}")
    print("-" * 54)
    print(f"{'Multi-Head Attention':<30} {attn_params:>12,} {100*attn_params/total_params:>11.1f}%")
    print(f"{'Feed-Forward Network':<30} {ffn_params:>12,} {100*ffn_params/total_params:>11.1f}%")
    print(f"{'Layer Norm 1':<30} {norm1_params:>12,} {100*norm1_params/total_params:>11.1f}%")
    print(f"{'Layer Norm 2':<30} {norm2_params:>12,} {100*norm2_params/total_params:>11.1f}%")
    print("-" * 54)
    print(f"{'TOTAL':<30} {total_params:>12,} {'100.0%':>12}")
    print()
    
    return {
        'attention': attn_params,
        'ffn': ffn_params,
        'norm': norm1_params + norm2_params,
        'total': total_params
    }

params = analyze_encoder_layer(encoder_layer)

# Visualize as pie chart
fig, ax = plt.subplots(figsize=(8, 6))
labels = ['Multi-Head Attention', 'Feed-Forward Network', 'Layer Normalization']
sizes = [params['attention'], params['ffn'], params['norm']]
colors = ['#ff9999', '#66b3ff', '#99ff99']
explode = (0, 0.05, 0)

ax.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',
       shadow=True, startangle=90)
ax.set_title('Encoder Layer Parameter Distribution', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("💡 Key Insight:")
print(f"   The FFN contains {100*params['ffn']/params['total']:.1f}% of the parameters!")
print("   This is because d_ff = 4 × d_model makes it very large.")

### **10. Testing with Masking (Padding Mask)**

In real applications, sequences often have different lengths and are padded to the same length. We need to **mask** the padding tokens so the model ignores them.

**Padding Mask:**
- 1 = real token (attend to it)
- 0 = padding token (ignore it)

```
Sequence: ["Hello", "world", "<PAD>", "<PAD>"]
Mask:     [   1,       1,        0,        0  ]
```

In [ ]:
# Create a function to generate padding masks

def create_padding_mask(seq_len, pad_positions):
    """
    Create a padding mask for attention.
    
    Args:
        seq_len: Total sequence length
        pad_positions: List of positions that are padding (to be masked)
    
    Returns:
        mask: Tensor of shape (1, 1, 1, seq_len) for broadcasting
    """
    mask = torch.ones(1, 1, 1, seq_len)
    for pos in pad_positions:
        mask[0, 0, 0, pos] = 0
    return mask

# Example: Sequence of length 10, with last 3 positions as padding
seq_len = 10
pad_positions = [7, 8, 9]  # Last 3 are padding

mask = create_padding_mask(seq_len, pad_positions)
print("Padding Mask:")
print(f"  Shape: {mask.shape}")
print(f"  Values: {mask.squeeze().numpy()}")
print(f"  (1 = attend, 0 = ignore)")
print()

# Test encoder layer with mask
x = torch.randn(1, seq_len, d_model)

encoder_layer.eval()
with torch.no_grad():
    output_masked, attn_weights_masked = encoder_layer(x, mask=mask)

print("Encoder output with masking:")
print(f"  Output shape: {output_masked.shape}")

# Visualize attention with masking
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Without mask (for comparison, re-run)
with torch.no_grad():
    _, attn_no_mask = encoder_layer(x, mask=None)

# Head 0 without mask
im1 = axes[0].imshow(attn_no_mask[0, 0].detach().numpy(), cmap='Blues', vmin=0, vmax=0.3)
axes[0].set_title('Head 1: Without Mask', fontsize=12)
axes[0].set_xlabel('Key Position')
axes[0].set_ylabel('Query Position')
plt.colorbar(im1, ax=axes[0])

# Head 0 with mask
im2 = axes[1].imshow(attn_weights_masked[0, 0].detach().numpy(), cmap='Blues', vmin=0, vmax=0.3)
axes[1].set_title('Head 1: With Padding Mask\n(Positions 7-9 masked)', fontsize=12)
axes[1].set_xlabel('Key Position')
axes[1].set_ylabel('Query Position')
axes[1].axvline(x=6.5, color='red', linestyle='--', linewidth=2, label='Mask boundary')
plt.colorbar(im2, ax=axes[1])

plt.suptitle('Effect of Padding Mask on Attention', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n✅ Notice how attention weights are ~0 for masked positions (7-9)!")
print("   The model correctly ignores padding tokens.")

### **11. Verifying Residual Connections**

In [ ]:
# Let's verify that residual connections help preserve information

# Create a simple test: if the sublayers output zeros, 
# the residual connection should preserve the input (after normalization)

class ZeroAttention(nn.Module):
    """A mock attention that outputs zeros (for testing residual connections)."""
    def forward(self, q, k, v, mask=None):
        return torch.zeros_like(q), None

class ZeroFFN(nn.Module):
    """A mock FFN that outputs zeros (for testing residual connections)."""
    def forward(self, x):
        return torch.zeros_like(x)

# Create encoder layer with zero sublayers
encoder_zero = EncoderLayer(d_model=64, num_heads=4, d_ff=256, dropout=0.0)
encoder_zero.self_attention = ZeroAttention()
encoder_zero.feed_forward = ZeroFFN()
encoder_zero.eval()

# Test input
x_test = torch.randn(1, 5, 64)

with torch.no_grad():
    output_zero, _ = encoder_zero(x_test)

# Compare input and output
# Due to layer normalization, values will be normalized but correlation should be high
correlation = torch.corrcoef(torch.stack([x_test.flatten(), output_zero.flatten()]))[0, 1]

print("=" * 60)
print("RESIDUAL CONNECTION VERIFICATION")
print("=" * 60)
print()
print("Test: When sublayers output zeros, residual preserves input")
print()
print(f"Input mean:  {x_test.mean().item():.4f}")
print(f"Output mean: {output_zero.mean().item():.4f} (normalized to ~0)")
print()
print(f"Input std:   {x_test.std().item():.4f}")
print(f"Output std:  {output_zero.std().item():.4f} (normalized to ~1)")
print()
print(f"Correlation: {correlation.item():.4f}")
print()
print("✅ High correlation shows residual connections preserve information!")
print("   Even with zero sublayer outputs, the input information flows through.")

### **12. Summary: The Complete Encoder Layer**

Excellent work! You've built the complete Encoder Layer! 🎉

**What We Built Today:**

✅ **Residual Connections**: Skip connections that enable gradient flow
- Formula: $\text{output} = x + \text{Sublayer}(x)$
- Prevents vanishing gradients in deep networks

✅ **Layer Normalization**: Stabilizes training by normalizing activations
- Normalizes across the feature dimension (not batch)
- Adds learnable scale ($\gamma$) and shift ($\beta$)

✅ **Complete Encoder Layer**: Assembled all components together
```
Input → Self-Attention → Add & Norm → FFN → Add & Norm → Output
```

**Key Takeaways:**

| Component | Role |
|-----------|------|
| Multi-Head Attention | Capture relationships between positions |
| Residual Connections | Enable gradient flow, preserve information |
| Layer Normalization | Stabilize training, normalize activations |
| Feed-Forward Network | Add non-linear transformation power |

---

**Our Progress:**

| Component | Status |
|-----------|--------|
| Multi-Head Attention | ✅ Day 7 |
| Positional Encoding | ✅ Step 1 |
| Feed-Forward Network | ✅ Step 2 |
| **Complete Encoder Layer** | ✅ **Step 3 (Today!)** |
| Encoder Stack | ⏳ Step 4 (Next!) |

**Next Up:** In Step 4, we'll stack multiple Encoder Layers to build the complete **Encoder Stack**!

### **13. Exercises**

Try these exercises to deepen your understanding!

**Exercise 1:** Implement a **Pre-Norm** version of the Encoder Layer (normalize before sublayer, not after).

**Exercise 2:** Add a parameter to toggle between Pre-Norm and Post-Norm variants.

**Exercise 3:** What happens if you remove the residual connections? Run an experiment and compare gradient magnitudes.

**Exercise 4:** Try using GELU activation instead of ReLU. Does it change the output distribution?

**Exercise 5:** Implement dropout on the attention weights (inside the attention mechanism). How does this affect the attention patterns?

In [ ]:
# Space for your exercises!

# Exercise 1: Pre-Norm Encoder Layer
# Hint: Apply LayerNorm BEFORE the sublayer, not after
# x = x + Sublayer(LayerNorm(x))

class PreNormEncoderLayer(nn.Module):
    """
    Pre-Norm variant of the Encoder Layer.
    Used in GPT-2, GPT-3, LLaMA, etc.
    """
    
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        # TODO: Implement Pre-Norm version
        pass
    
    def forward(self, x, mask=None):
        # TODO: Implement forward pass
        # Remember: LayerNorm comes BEFORE the sublayer!
        pass

# Your code here!